In [1]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd())) 

import pandas as pd
from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
from features.load_team_stats_from_db import load_team_stats_from_db
from features.load_games_from_db import load_games_from_db

In [3]:
df_g = load_games_from_db()
df_ts = load_team_stats_from_db()

2025-06-29 16:08:25,151 [INFO] Successully loaded 10641 games from database


There are 10641 games for  modeling...


2025-06-29 16:08:25,408 [INFO] Successully loaded 25560 rows from database


There are 25560 team-games for  modeling...


In [5]:
df_g.head()

,game_id,game_date,game_date_time,home_team_id,away_team_id,home_team,away_team,home_score,away_score,state,venue,game_type
0,634642,2021-04-01,2021-04-01 17:05:00+00:00,147,141,New York Yankees,Toronto Blue Jays,2,3,Final,Yankee Stadium,R
1,634645,2021-04-01,2021-04-01 17:10:00+00:00,116,114,Detroit Tigers,Cleveland Indians,3,2,Final,Comerica Park,R
2,634638,2021-04-01,2021-04-01 18:10:00+00:00,158,142,Milwaukee Brewers,Minnesota Twins,6,5,Final,American Family Field,R
3,634634,2021-04-01,2021-04-01 18:20:00+00:00,112,134,Chicago Cubs,Pittsburgh Pirates,3,5,Final,Wrigley Field,R
4,634622,2021-04-01,2021-04-01 19:05:00+00:00,143,144,Philadelphia Phillies,Atlanta Braves,3,2,Final,Citizens Bank Park,R


In [6]:
df_ts.head()

,game_pk,team_side,team_id,runs_batting,hits_batting,strikeouts_batting,baseonballs_batting,avg,obp,slg,pitchesthrown,balls_pitching,strikes_pitching,strikeouts_pitching,baseonballs_pitching,hits_pitching,earnedruns,homeruns_pitching,runs_pitching,era,whip,groundouts_pitching,airouts_pitching,total,putouts,assists,errors,doubleplays,tripleplays,rangefactor,caughtstealing,passedball,innings
0,634642,home,147,2,6,13,6,0.171,0.293,0.257,159,55,104,13,2,8,2,1,3,1.80,1.00,4,11,0,30,7,0,0,0,0.0,1,0,0.0
1,634642,away,141,3,8,13,2,0.222,0.263,0.333,181,78,103,13,6,6,2,1,2,1.80,1.20,10,6,0,30,10,0,0,0,0.0,0,0,0.0
2,634645,home,116,3,5,14,5,0.167,0.286,0.300,154,60,94,5,6,6,2,1,2,2.00,1.33,11,10,0,27,12,0,0,0,0.0,0,0,0.0
3,634645,away,114,2,6,5,6,0.188,0.316,0.313,125,48,77,14,5,5,3,1,3,3.38,1.25,5,6,0,24,6,1,0,0,0.0,0,0,0.0
4,634638,home,158,6,11,9,2,0.275,0.341,0.300,189,74,115,17,7,10,5,1,5,4.50,1.70,7,6,0,30,6,0,0,0,0.0,0,0,0.0


In [ ]:
df.isna().sum()

In [ ]:
df[df['era'].isna()]

In [ ]:
df[df['pitchesthrown']<=1]

In [80]:
common_game_cols = ['game_id','game_date','game_date_time']
home_game_cols = common_game_cols + ['home_team_id']
away_game_cols = common_game_cols + ['away_team_id']

# Merge Strategy
# 1. Split Home and Away
# 2. Merge separately
# 3. Concatenate and sort

# 1. Split
df_ts_home = df_ts.query("team_side=='home'").copy()
df_ts_away = df_ts.query("team_side=='away'").copy()
df_g['game_date'] = pd.to_datetime(df_g['game_date'])
df_g['game_date_time'] = pd.to_datetime(df_g['game_date_time'])

# 2. Merge separately
# Home Merge
df_ts_home = df_ts_home.merge(
    df_g[home_game_cols], 
    how='left', 
    left_on=['game_pk','team_id'],
    right_on=['game_id','home_team_id']
)
df_ts_home.drop(['home_team_id'], axis=1, inplace=True)
print(f"df_ts_home: {df_ts_home.shape}")
print(display(df_ts_home.head(3)))
# Away Merge
df_ts_away = df_ts_away.merge(
    df_g[away_game_cols],
    how='left',
    left_on=['game_pk','team_id'],
    right_on=['game_id','away_team_id']
)
df_ts_away.drop(['away_team_id'], axis=1, inplace=True)
print(f"df_ts_away: {df_ts_away.shape}")
print(display(df_ts_away.head(3)))

# 3. Concatenate and sort
df_ts_c = pd.concat([df_ts_home, df_ts_away], ignore_index=True) \
    .sort_values(['team_id','game_date_time']) \
    .reset_index(drop=True)

# 4. Drop rows with no matching game_id from games dataset
df_ts_c = df_ts_c[~df_ts_c['game_id'].isna()]

# 5. Add season
df_ts_c['season'] = df_ts_c['game_date'].dt.year.astype('int64')

df_ts_home: (12780, 36)


,game_pk,team_side,team_id,runs_batting,hits_batting,strikeouts_batting,baseonballs_batting,avg,obp,slg,pitchesthrown,balls_pitching,strikes_pitching,strikeouts_pitching,baseonballs_pitching,hits_pitching,earnedruns,homeruns_pitching,runs_pitching,era,whip,groundouts_pitching,airouts_pitching,total,putouts,assists,errors,doubleplays,tripleplays,rangefactor,caughtstealing,passedball,innings,game_id,game_date,game_date_time
0,634642,home,147,2,6,13,6,0.171,0.293,0.257,159,55,104,13,2,8,2,1,3,1.8,1.00,4,11,0,30,7,0,0,0,0.0,1,0,0.0,634642.0,2021-04-01,2021-04-01 17:05:00+00:00
1,634645,home,116,3,5,14,5,0.167,0.286,0.300,154,60,94,5,6,6,2,1,2,2.0,1.33,11,10,0,27,12,0,0,0,0.0,0,0,0.0,634645.0,2021-04-01,2021-04-01 17:10:00+00:00
2,634638,home,158,6,11,9,2,0.275,0.341,0.300,189,74,115,17,7,10,5,1,5,4.5,1.70,7,6,0,30,6,0,0,0,0.0,0,0,0.0,634638.0,2021-04-01,2021-04-01 18:10:00+00:00


None
df_ts_away: (12780, 36)


,game_pk,team_side,team_id,runs_batting,hits_batting,strikeouts_batting,baseonballs_batting,avg,obp,slg,pitchesthrown,balls_pitching,strikes_pitching,strikeouts_pitching,baseonballs_pitching,hits_pitching,earnedruns,homeruns_pitching,runs_pitching,era,whip,groundouts_pitching,airouts_pitching,total,putouts,assists,errors,doubleplays,tripleplays,rangefactor,caughtstealing,passedball,innings,game_id,game_date,game_date_time
0,634642,away,141,3,8,13,2,0.222,0.263,0.333,181,78,103,13,6,6,2,1,2,1.80,1.20,10,6,0,30,10,0,0,0,0.0,0,0,0.0,634642.0,2021-04-01,2021-04-01 17:05:00+00:00
1,634645,away,114,2,6,5,6,0.188,0.316,0.313,125,48,77,14,5,5,3,1,3,3.38,1.25,5,6,0,24,6,1,0,0,0.0,0,0,0.0,634645.0,2021-04-01,2021-04-01 17:10:00+00:00
2,634638,away,142,5,10,17,7,0.256,0.370,0.436,151,50,101,9,2,11,1,0,6,1.00,1.44,10,10,0,27,10,2,0,0,0.0,0,0,0.0,634638.0,2021-04-01,2021-04-01 18:10:00+00:00


None


In [81]:
df_ts_c.shape

(21282, 37)

In [38]:
df_ts_c.head(5)

,game_pk,team_side,team_id,runs_batting,hits_batting,strikeouts_batting,baseonballs_batting,avg,obp,slg,pitchesthrown,balls_pitching,strikes_pitching,strikeouts_pitching,baseonballs_pitching,hits_pitching,earnedruns,homeruns_pitching,runs_pitching,era,whip,groundouts_pitching,airouts_pitching,total,putouts,assists,errors,doubleplays,tripleplays,rangefactor,caughtstealing,passedball,innings,game_id,game_date,game_date_time,season
10,634655,home,108,4,6,10,4,0.200,0.294,0.300,125,42,83,10,2,7,3,1,3,3.00,1.00,10,6,0,27,10,0,0,0,0.0,1,0,0.0,634655.0,2021-04-01,2021-04-02 02:05:00+00:00,2021
17,634631,home,108,8,8,12,5,0.215,0.311,0.400,173,52,121,11,5,13,12,2,12,7.50,1.50,11,5,0,27,12,1,0,0,0.0,0,0,0.0,634631.0,2021-04-02,2021-04-03 01:38:00+00:00,2021
33,634646,home,108,5,9,11,2,0.235,0.312,0.408,144,47,97,10,3,9,3,1,3,6.00,1.44,13,3,0,27,15,0,0,0,0.0,0,0,0.0,634646.0,2021-04-03,2021-04-04 01:07:00+00:00,2021
47,634637,home,108,7,10,5,5,0.252,0.336,0.450,163,65,98,12,6,5,1,0,4,4.75,1.39,11,6,0,27,9,3,0,0,0.0,0,1,0.0,634637.0,2021-04-04,2021-04-05 00:37:00+00:00,2021
58,634552,home,108,7,10,3,5,0.261,0.347,0.436,190,78,112,9,5,11,6,1,6,5.00,1.47,13,6,0,27,13,0,0,0,0.0,0,0,0.0,634552.0,2021-04-05,2021-04-06 01:38:00+00:00,2021


In [20]:
df_ts_c[df_ts_c['game_id'].isna()].head()

,game_pk,team_side,team_id,runs_batting,hits_batting,strikeouts_batting,baseonballs_batting,avg,obp,slg,pitchesthrown,balls_pitching,strikes_pitching,strikeouts_pitching,baseonballs_pitching,hits_pitching,earnedruns,homeruns_pitching,runs_pitching,era,whip,groundouts_pitching,airouts_pitching,total,putouts,assists,errors,doubleplays,tripleplays,rangefactor,caughtstealing,passedball,innings,game_id,game_date,game_date_time
21424,764388,away,105,8,7,11,8,0.206,0.386,0.265,146,63,83,3,8,4,1,0,1,1.00,1.33,11,10,0,27,18,0,0,0,0.0,0,0,0.0,NaN,NaN,NaT
24335,789325,away,105,4,6,9,7,0.182,0.325,0.242,172,60,112,9,6,9,2,0,3,2.00,1.67,6,11,0,27,6,2,0,0,0.0,1,0,0.0,NaN,NaN,NaT
221,634465,home,108,7,10,3,2,0.250,0.311,0.419,80,22,58,6,0,4,1,0,1,5.15,1.44,9,5,0,21,7,0,0,0,0.0,0,0,0.0,NaN,NaN,NaT
236,634441,home,108,3,3,12,2,0.248,0.309,0.417,150,62,88,6,8,6,6,3,6,5.20,1.45,8,7,0,21,7,0,0,0,0.0,0,0,0.0,NaN,NaN,NaT
2504,706942,home,108,12,19,6,2,0.447,0.500,0.803,63,10,53,8,2,12,5,1,5,7.94,1.76,7,10,0,27,7,0,0,0,0.0,0,0,0.0,NaN,NaN,NaT


In [25]:
df_ts_c.isna().sum()

game_pk                 0
team_side               0
team_id                 0
runs_batting            0
hits_batting            0
strikeouts_batting      0
baseonballs_batting     0
avg                     0
obp                     0
slg                     0
pitchesthrown           0
balls_pitching          0
strikes_pitching        0
strikeouts_pitching     0
baseonballs_pitching    0
hits_pitching           0
earnedruns              0
homeruns_pitching       0
runs_pitching           0
era                     0
whip                    0
groundouts_pitching     0
airouts_pitching        0
total                   0
putouts                 0
assists                 0
errors                  0
doubleplays             0
tripleplays             0
rangefactor             0
caughtstealing          0
passedball              0
innings                 0
game_id                 0
game_date               0
game_date_time          0
dtype: int64

In [45]:
df_ts_c[['game_pk','team_side','team_id','game_id','game_date','game_date_time', 'runs_batting']].head(20)

,game_pk,team_side,team_id,game_id,game_date,game_date_time,runs_batting
10,634655,home,108,634655.0,2021-04-01,2021-04-02 02:05:00+00:00,4
17,634631,home,108,634631.0,2021-04-02,2021-04-03 01:38:00+00:00,8
33,634646,home,108,634646.0,2021-04-03,2021-04-04 01:07:00+00:00,5
47,634637,home,108,634637.0,2021-04-04,2021-04-05 00:37:00+00:00,7
58,634552,home,108,634552.0,2021-04-05,2021-04-06 01:38:00+00:00,7
65,634586,home,108,634586.0,2021-04-06,2021-04-06 20:07:00+00:00,2
12878,634545,away,108,634545.0,2021-04-08,2021-04-08 23:07:00+00:00,7
12883,634512,away,108,634512.0,2021-04-09,2021-04-09 23:07:00+00:00,7
12899,632227,away,108,632227.0,2021-04-10,2021-04-10 23:07:00+00:00,1
12925,632197,away,108,632197.0,2021-04-12,2021-04-13 00:10:00+00:00,10


In [40]:
df_ts_c.query("game_pk==634655")

,game_pk,team_side,team_id,runs_batting,hits_batting,strikeouts_batting,baseonballs_batting,avg,obp,slg,pitchesthrown,balls_pitching,strikes_pitching,strikeouts_pitching,baseonballs_pitching,hits_pitching,earnedruns,homeruns_pitching,runs_pitching,era,whip,groundouts_pitching,airouts_pitching,total,putouts,assists,errors,doubleplays,tripleplays,rangefactor,caughtstealing,passedball,innings,game_id,game_date,game_date_time,season
10,634655,home,108,4,6,10,4,0.200,0.294,0.300,125,42,83,10,2,7,3,1,3,3.00,1.00,10,6,0,27,10,0,0,0,0.0,1,0,0.0,634655.0,2021-04-01,2021-04-02 02:05:00+00:00,2021
12790,634655,away,145,3,7,10,2,0.212,0.257,0.364,140,48,92,10,4,6,2,1,4,2.25,1.25,7,7,0,24,7,1,0,0,0.0,0,1,0.0,634655.0,2021-04-01,2021-04-02 02:05:00+00:00,2021


In [88]:
def rolling_avg_team_stat(df: pd.DataFrame, team_stat: str, window: int) -> pd.Series:
    return (
        df
        .groupby(['team_id','season'])[team_stat]
        .rolling(window=window, min_periods=1, closed='left')
        .mean()
        .fillna(value=0)
        .reset_index(drop=True)
    )

def rolling_std_team_stat(df: pd.DataFrame, team_stat: str, window: int) -> pd.Series:
    return (
        df
        .groupby(['team_id','season'])[team_stat]
        .rolling(window=window, min_periods=1, closed='left')
        .std()
        .fillna(value=0)
        .reset_index(drop=True)
    )


In [65]:
rolling_cols = ['obp','slg','era','whip','strikeouts_pitching','baseonballs_pitching','hits_pitching']
rb = rolling_avg_team_stat(df=df_ts_c, team_stat='runs_batting', window=5)

In [89]:
def get_rolling_team_stat_features(df: pd.DataFrame, team_stat_cols: list[str], window: list[int] =[7]) -> pd.DataFrame:
    df = df.copy().reset_index(drop=True)
    for stat in team_stat_cols:
        for w in window:
            df[f"rolling_avg_{stat}_{w}days"] = rolling_avg_team_stat(df, team_stat=stat, window=w)
            df[f"rolling_std_{stat}_{w}days"] = rolling_std_team_stat(df, team_stat=stat, window=w)

    return df



In [90]:
df1 = get_rolling_team_stat_features(df_ts_c, team_stat_cols=rolling_cols, window=[7,14])


In [72]:
df_ts_c.shape, df1.shape

((21282, 37), (21282, 65))

In [91]:
df1.head()

,game_pk,team_side,team_id,runs_batting,hits_batting,strikeouts_batting,baseonballs_batting,avg,obp,slg,pitchesthrown,balls_pitching,strikes_pitching,strikeouts_pitching,baseonballs_pitching,hits_pitching,earnedruns,homeruns_pitching,runs_pitching,era,whip,groundouts_pitching,airouts_pitching,total,putouts,assists,errors,doubleplays,tripleplays,rangefactor,caughtstealing,passedball,innings,game_id,game_date,game_date_time,season,rolling_avg_obp_7days,rolling_std_obp_7days,rolling_avg_obp_14days,rolling_std_obp_14days,rolling_avg_slg_7days,rolling_std_slg_7days,rolling_avg_slg_14days,rolling_std_slg_14days,rolling_avg_era_7days,rolling_std_era_7days,rolling_avg_era_14days,rolling_std_era_14days,rolling_avg_whip_7days,rolling_std_whip_7days,rolling_avg_whip_14days,rolling_std_whip_14days,rolling_avg_strikeouts_pitching_7days,rolling_std_strikeouts_pitching_7days,rolling_avg_strikeouts_pitching_14days,rolling_std_strikeouts_pitching_14days,rolling_avg_baseonballs_pitching_7days,rolling_std_baseonballs_pitching_7days,rolling_avg_baseonballs_pitching_14days,rolling_std_baseonballs_pitching_14days,rolling_avg_hits_pitching_7days,rolling_std_hits_pitching_7days,rolling_avg_hits_pitching_14days,rolling_std_hits_pitching_14days
0,634655,home,108,4,6,10,4,0.200,0.294,0.300,125,42,83,10,2,7,3,1,3,3.00,1.00,10,6,0,27,10,0,0,0,0.0,1,0,0.0,634655.0,2021-04-01,2021-04-02 02:05:00+00:00,2021,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,634631,home,108,8,8,12,5,0.215,0.311,0.400,173,52,121,11,5,13,12,2,12,7.50,1.50,11,5,0,27,12,1,0,0,0.0,0,0,0.0,634631.0,2021-04-02,2021-04-03 01:38:00+00:00,2021,0.294000,0.000000,0.294000,0.000000,0.300000,0.000000,0.300000,0.000000,3.0000,0.000000,3.0000,0.000000,1.000000,0.000000,1.000000,0.000000,10.000000,0.000000,10.000000,0.000000,2.000000,0.000000,2.000000,0.000000,7.000000,0.000000,7.000000,0.000000
2,634646,home,108,5,9,11,2,0.235,0.312,0.408,144,47,97,10,3,9,3,1,3,6.00,1.44,13,3,0,27,15,0,0,0,0.0,0,0,0.0,634646.0,2021-04-03,2021-04-04 01:07:00+00:00,2021,0.302500,0.012021,0.302500,0.012021,0.350000,0.070711,0.350000,0.070711,5.2500,3.181981,5.2500,3.181981,1.250000,0.353553,1.250000,0.353553,10.500000,0.707107,10.500000,0.707107,3.500000,2.121320,3.500000,2.121320,10.000000,4.242641,10.000000,4.242641
3,634637,home,108,7,10,5,5,0.252,0.336,0.450,163,65,98,12,6,5,1,0,4,4.75,1.39,11,6,0,27,9,3,0,0,0.0,0,1,0.0,634637.0,2021-04-04,2021-04-05 00:37:00+00:00,2021,0.305667,0.010116,0.305667,0.010116,0.369333,0.060178,0.369333,0.060178,5.5000,2.291288,5.5000,2.291288,1.313333,0.273008,1.313333,0.273008,10.333333,0.577350,10.333333,0.577350,3.333333,1.527525,3.333333,1.527525,9.666667,3.055050,9.666667,3.055050
4,634552,home,108,7,10,3,5,0.261,0.347,0.436,190,78,112,9,5,11,6,1,6,5.00,1.47,13,6,0,27,13,0,0,0,0.0,0,0,0.0,634552.0,2021-04-05,2021-04-06 01:38:00+00:00,2021,0.313250,0.017270,0.313250,0.017270,0.389500,0.063569,0.389500,0.063569,5.3125,1.908042,5.3125,1.908042,1.332500,0.226182,1.332500,0.226182,10.750000,0.957427,10.750000,0.957427,4.000000,1.825742,4.000000,1.825742,8.500000,3.415650,8.500000,3.415650


In [92]:
print(df1[['game_pk','team_side','team_id','game_date','game_date_time','obp','rolling_avg_obp_7days','rolling_std_obp_7days']].head())

   game_pk team_side  team_id  game_date            game_date_time    obp  \
0   634655      home      108 2021-04-01 2021-04-02 02:05:00+00:00  0.294   
1   634631      home      108 2021-04-02 2021-04-03 01:38:00+00:00  0.311   
2   634646      home      108 2021-04-03 2021-04-04 01:07:00+00:00  0.312   
3   634637      home      108 2021-04-04 2021-04-05 00:37:00+00:00  0.336   
4   634552      home      108 2021-04-05 2021-04-06 01:38:00+00:00  0.347   

   rolling_avg_obp_7days  rolling_std_obp_7days  
0               0.000000               0.000000  
1               0.294000               0.000000  
2               0.302500               0.012021  
3               0.305667               0.010116  
4               0.313250               0.017270  


In [93]:
df1.head()

,game_pk,team_side,team_id,runs_batting,hits_batting,strikeouts_batting,baseonballs_batting,avg,obp,slg,pitchesthrown,balls_pitching,strikes_pitching,strikeouts_pitching,baseonballs_pitching,hits_pitching,earnedruns,homeruns_pitching,runs_pitching,era,whip,groundouts_pitching,airouts_pitching,total,putouts,assists,errors,doubleplays,tripleplays,rangefactor,caughtstealing,passedball,innings,game_id,game_date,game_date_time,season,rolling_avg_obp_7days,rolling_std_obp_7days,rolling_avg_obp_14days,rolling_std_obp_14days,rolling_avg_slg_7days,rolling_std_slg_7days,rolling_avg_slg_14days,rolling_std_slg_14days,rolling_avg_era_7days,rolling_std_era_7days,rolling_avg_era_14days,rolling_std_era_14days,rolling_avg_whip_7days,rolling_std_whip_7days,rolling_avg_whip_14days,rolling_std_whip_14days,rolling_avg_strikeouts_pitching_7days,rolling_std_strikeouts_pitching_7days,rolling_avg_strikeouts_pitching_14days,rolling_std_strikeouts_pitching_14days,rolling_avg_baseonballs_pitching_7days,rolling_std_baseonballs_pitching_7days,rolling_avg_baseonballs_pitching_14days,rolling_std_baseonballs_pitching_14days,rolling_avg_hits_pitching_7days,rolling_std_hits_pitching_7days,rolling_avg_hits_pitching_14days,rolling_std_hits_pitching_14days
0,634655,home,108,4,6,10,4,0.200,0.294,0.300,125,42,83,10,2,7,3,1,3,3.00,1.00,10,6,0,27,10,0,0,0,0.0,1,0,0.0,634655.0,2021-04-01,2021-04-02 02:05:00+00:00,2021,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,634631,home,108,8,8,12,5,0.215,0.311,0.400,173,52,121,11,5,13,12,2,12,7.50,1.50,11,5,0,27,12,1,0,0,0.0,0,0,0.0,634631.0,2021-04-02,2021-04-03 01:38:00+00:00,2021,0.294000,0.000000,0.294000,0.000000,0.300000,0.000000,0.300000,0.000000,3.0000,0.000000,3.0000,0.000000,1.000000,0.000000,1.000000,0.000000,10.000000,0.000000,10.000000,0.000000,2.000000,0.000000,2.000000,0.000000,7.000000,0.000000,7.000000,0.000000
2,634646,home,108,5,9,11,2,0.235,0.312,0.408,144,47,97,10,3,9,3,1,3,6.00,1.44,13,3,0,27,15,0,0,0,0.0,0,0,0.0,634646.0,2021-04-03,2021-04-04 01:07:00+00:00,2021,0.302500,0.012021,0.302500,0.012021,0.350000,0.070711,0.350000,0.070711,5.2500,3.181981,5.2500,3.181981,1.250000,0.353553,1.250000,0.353553,10.500000,0.707107,10.500000,0.707107,3.500000,2.121320,3.500000,2.121320,10.000000,4.242641,10.000000,4.242641
3,634637,home,108,7,10,5,5,0.252,0.336,0.450,163,65,98,12,6,5,1,0,4,4.75,1.39,11,6,0,27,9,3,0,0,0.0,0,1,0.0,634637.0,2021-04-04,2021-04-05 00:37:00+00:00,2021,0.305667,0.010116,0.305667,0.010116,0.369333,0.060178,0.369333,0.060178,5.5000,2.291288,5.5000,2.291288,1.313333,0.273008,1.313333,0.273008,10.333333,0.577350,10.333333,0.577350,3.333333,1.527525,3.333333,1.527525,9.666667,3.055050,9.666667,3.055050
4,634552,home,108,7,10,3,5,0.261,0.347,0.436,190,78,112,9,5,11,6,1,6,5.00,1.47,13,6,0,27,13,0,0,0,0.0,0,0,0.0,634552.0,2021-04-05,2021-04-06 01:38:00+00:00,2021,0.313250,0.017270,0.313250,0.017270,0.389500,0.063569,0.389500,0.063569,5.3125,1.908042,5.3125,1.908042,1.332500,0.226182,1.332500,0.226182,10.750000,0.957427,10.750000,0.957427,4.000000,1.825742,4.000000,1.825742,8.500000,3.415650,8.500000,3.415650


In [100]:
# Home merge
add_cols = ['rolling_avg_obp_7days']
df_g2 = df_g.merge(
    df1[['game_pk','team_id','game_date_time']+add_cols],
    how='left',
    left_on=['home_team_id','game_date_time'],
    right_on=['team_id','game_date_time']
)
df_g2.drop('team_id', axis=1, inplace=True)
home_cols_rename = {col: f'home_{col}' for col in add_cols}
df_g2.rename(columns=home_cols_rename, inplace=True)
# Away merge

In [110]:
team_stat_cols = ['obp','slg','era','whip','strikeouts_pitching','baseonballs_pitching','hits_pitching']
window=[7,14]
rolling_avg_team_stat_cols = [f"rolling_avg_{stat}_{w}days" for stat in team_stat_cols for w in window]

In [112]:
rolling_avg_team_stat_cols = [f"rolling_avg_{stat}_{w}days" for stat in team_stat_cols for w in window]
rolling_std_team_stat_cols = [f"rolling_std_{stat}_{w}days" for stat in team_stat_cols for w in window]
rolling_team_stat_cols = rolling_avg_team_stat_cols + rolling_std_team_stat_cols

In [113]:
rolling_team_stat_cols

['rolling_avg_obp_7days',
 'rolling_avg_obp_14days',
 'rolling_avg_slg_7days',
 'rolling_avg_slg_14days',
 'rolling_avg_era_7days',
 'rolling_avg_era_14days',
 'rolling_avg_whip_7days',
 'rolling_avg_whip_14days',
 'rolling_avg_strikeouts_pitching_7days',
 'rolling_avg_strikeouts_pitching_14days',
 'rolling_avg_baseonballs_pitching_7days',
 'rolling_avg_baseonballs_pitching_14days',
 'rolling_avg_hits_pitching_7days',
 'rolling_avg_hits_pitching_14days',
 'rolling_std_obp_7days',
 'rolling_std_obp_14days',
 'rolling_std_slg_7days',
 'rolling_std_slg_14days',
 'rolling_std_era_7days',
 'rolling_std_era_14days',
 'rolling_std_whip_7days',
 'rolling_std_whip_14days',
 'rolling_std_strikeouts_pitching_7days',
 'rolling_std_strikeouts_pitching_14days',
 'rolling_std_baseonballs_pitching_7days',
 'rolling_std_baseonballs_pitching_14days',
 'rolling_std_hits_pitching_7days',
 'rolling_std_hits_pitching_14days']